Author: Nicholas Orgel

Creation Date: 02.01.2023

# Task
Clean the files & combine them into one final DataFrame.

- This dataframe should have the following columns:
    - Hero (Just the name of the Hero)
    - Publisher
    - Gender
    - Eye Color
    - Race
    - Hair Color
    - Height (numeric)
    - Skin Color
    - Alignment
    - Weight (numeric)
    
    - **Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:**
        - Agility
        - Flight
        - Superspeed
        - etc.
        
---

**Hint: There is a space in "100 kg" or "52.5 cm"**

## Questions To Answer

**II: Use your combined DataFrame to answer the following questions.**

- 1. Compare the average weight of superheroes who have Super Speed to those who do not.
- 2. What is the average height of heroes for each publisher?

---

# Steps

## Import Libraries

## Load DataFrames